In [61]:
import mbuild as mb
import os
from parmed import load_file
import re
from mbuild.lib.atoms import H
import numpy as np
import random
import pubchempy as pcp
from rdkit import Chem 
from Main_functions.crosslink_function import *
from IPython.display import Image, display

In this case we will use the crosslinking of a polyamide EPON 862 with the curing Agent DETDA
https://pubs.acs.org/doi/full/10.1021/ma2005519 



<img src="images/EPON862.png">


In [62]:
Repeat_unit_from_file = mb.load("files/EPON862_RU.mol2")
CR_COMPOUND = mb.load("files/EPON862_Curing_Agent_1.mol2")

/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")
/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


We first load the repeat unit of the polymer, where a hydrogen in the C-NH2-C has been replaced by a Fluorine dummy atom

In [63]:
Repeat_unit_from_file.visualize(show_ports = True)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [64]:
ports_to_add_Dummy_atoms = {
    "CH2O-B" : [
        ["C",  # The first atom is the "anchor" or base atom 
          "B", "H", "H", "O"], # Bonded atoms to Carbon 
        ["B"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],

     "NH2C-B" : [
        ["N", # The first atom is the "anchor" or base atom 
          "B", "F", "H", "C"], # Bonded atoms to Carbon 
        ["B"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],
}

In [65]:
compound_1_port_DA = Add_port_CR(Repeat_unit_from_file,
                                type = "CH2O-B",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms)    
    
print("Added ports to the molecule")
tmp_cmp_DA = mb.clone(compound_1_port_DA)

#Adding second port to the repeat unit        
compound_with_2_ports_DA = Add_port_CR(tmp_cmp_DA,
                                    type = "NH2C-B",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms)      
print("Added 2 ports to the molecule")









Added ports to the molecule

Added 2 ports to the molecule


In [66]:
compound_with_2_ports_DA.visualize(show_ports = True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [67]:
repeat_units = 6 #Repeat Units in polymer chain
polymer_compound_ = mb.recipes.Polymer(monomers=[compound_with_2_ports_DA])
polymer_compound_.build( n=repeat_units,)
polymer_compound_.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Here we will define the crosslinker first as the EPON-862 monomer with 
dummy atoms Fluorine inplace of Hydrogen

In [68]:
CR_COMPOUND.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [69]:

RU = 5
N_chains = 5
n_sites = 1

#Repeat_unit_from_file = mb.load("files/Repeat_Unit_2.mol2")
dummy_atom_dictionary = {
                         "F": "H",}


box_system, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
#box_system = Crosslink_Pipeline(
    main_name = "EPON",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    Pre_existing_repeat_unit =True,
    Repeat_unit_compound_with_ports = compound_with_2_ports_DA, # Here we pass already the repeat unit with ports

    Dummy_atoms = True,
    Dummy_atom_dictionary = dummy_atom_dictionary,

    
    repeat_units = 5,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 3.0,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                              #Crosslinker details
    crosslinker_name = "BASE",                                # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports
    Crosslinker_compound = CR_COMPOUND,
    

 
                                                              #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 40,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "N",
                            "target_atom" : "F",
                            "bonded_atoms": ["C", "H", "F"], 
                            "anchor_atom_n_bonds" : 4,
                            },
                            
                          
                            ],
    
    
    crosslinker_backbone_target_list = [
                        {
                        "anchor_atom" : "C",
                        "target_atom" : ["S"],
                        "anchor_atom_n_bonds" : 4,
                        "extra_pruning": [],
                        "exclude": [],
                        },
                        
                    
    ],
    
    prune_extra_flag =False,                               # No pruning extra atoms 
    crosslink_sites_per_molecule = 1 ,                     # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
   
    

        
    
    
)


for suffix in ["mol2", "pdb"]:
    box_system.save(f"output/5_3_EPON-862_DETDA-Short.{suffix}",overwrite = True)


Number of ports per chain 2
This values are the number of crosslinking monomers that will be bonded between each pair of chains to attain the desire degree of crosslinking
{1: 26, 2: 53}
To attain the degree of crosslinking 40% 
 4 crosslinkers are needed 
 2 between each polymer chain
The actual attainable value of crosslinking is 53%
Which is the highest crosslinking percentage attainable
Repeat unit passed to function
Done with loading compounds
{'POL-0': [(0, array([1.42768002, 0.01147   , 0.20259   ])), (1, array([ 2.47937989, -0.18545   ,  0.56428999])), (2, array([ 6.32626118, -0.58465415,  0.6656002 ])), (3, array([ 7.38292086, -0.83222896,  0.97841648])), (4, array([11.1957288 , -1.48200436,  0.88670552])), (5, array([12.25119201, -1.78294811,  1.15336501])), (6, array([16.00259953, -2.69023925,  0.88524613])), (7, array([17.05072264, -3.04670129,  1.10896418])), (8, array([20.71405245, -4.2157275 ,  0.6829175 ])), (9, array([21.74876952, -4.62926965,  0.86736388]))], 'POL-1':

In [70]:
box_system.visualize()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

At the same time the crosslinker which in the first part was the monomer, the chemistry of the crosslinker can be further modified to create a more complex polymer
network.
This can be achieved by creating a polymer that's going to act this time as the crosslinker, we can leverage Mbuild's Polymer class
to create a co-polymer with the existing chemistries:

Block A

In [71]:
BLOCK_A = mb.clone(CR_COMPOUND)

In [72]:
BLOCK_A.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [73]:
ports_to_add_Dummy_atoms_COP_A = {
    "CH2-S" : [
        ["C",  # The first atom is the "anchor" or base atom 
          "S", "H", "H", "C"], # Bonded atoms to Carbon 
        ["S"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],


}

compound_1_port_DA = Add_port_CR(BLOCK_A,
                                type = "CH2-S",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms_COP_A)    
    
print("Added ports to the molecule")
tmp_cmp_BLOCK_A = mb.clone(compound_1_port_DA)

#Adding second port to the repeat unit        
compound_with_2_ports_BLOCK_A = Add_port_CR(tmp_cmp_BLOCK_A,
                                    type = "CH2-S",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms_COP_A)      
print("Added 2 ports to the molecule")





Added ports to the molecule



Added 2 ports to the molecule


In [74]:
compound_with_2_ports_BLOCK_A.visualize(show_ports=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [75]:
Block_B = mb.load("files/EPON862_Curing_Agent_2.mol2")
Block_B.visualize()

/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [76]:
ports_to_add_Dummy_atoms_COP_B = {
    "NH2-S" : [
        ["N",  # The first atom is the "anchor" or base atom 
          "S", "H", "H", "C"], # Bonded atoms to Carbon 
        ["S"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],


}

compound_1_port_B = Add_port_CR(Block_B,
                                type = "NH2-S",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms_COP_B)    
    
print("Added ports to the molecule")
tmp_cmp_BLOCK_B = mb.clone(compound_1_port_B)

#Adding second port to the repeat unit        
compound_with_2_ports_BLOCK_B = Add_port_CR(tmp_cmp_BLOCK_B,
                                    type = "NH2-S",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms_COP_B)      
print("Added 2 ports to the molecule")

Added ports to the molecule
Added 2 ports to the molecule


In [77]:
compound_with_2_ports_BLOCK_B.visualize(show_ports=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [78]:
Crosslinker_polymer = mb.recipes.Polymer(monomers=[compound_with_2_ports_BLOCK_A, compound_with_2_ports_BLOCK_B ])
Crosslinker_polymer.build( n=1, sequence="ABABA")
Crosslinker_polymer.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The next step is using the dummy atoms in order to facilitate the crosslinking process, to do this the support function:
 Replace_Atom_with_dummy -> will go through the particles of the created polymer that will act as crosslinker:
 1. Look for the anchor atom C (Carbon)
 2. Match the carbon that has 3 Hydrogens and is bonded to another Carbon (e.g. terminal methyl)
 3. Verify that this methyl is bonded to another carbon that is bonded to two oxygens (sanity check to avoid multiple matches)
 4. replace one hydrogen in the terminal methyl group with a sulfur atom (dummy atom)

In [79]:
Dummy_Crosslinker_Polymer = Replace_Atom_with_dummy(Compound = Crosslinker_polymer,
                            replacement_atom = "S",
                            anchor_name = "C",
                            list_atoms_bonded_anchor = ["H", "H", "H", "C"],
                            neighbor_anchor_name = "C",
                            list_atoms_bonded_neighbor_anchor = ["O", "O", "C", "H"],
                            )

Found one
Found one


In [80]:
Dummy_Crosslinker_Polymer.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now we create a new crosslinked polymer with the new crosslinker molecule

In [81]:

RU = 5
N_chains = 5
n_sites = 1

#Repeat_unit_from_file = mb.load("files/Repeat_Unit_2.mol2")
dummy_atom_dictionary = {
                         "F": "H",}


box_system, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
#box_system = Crosslink_Pipeline(
    main_name = "EPON",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    Pre_existing_repeat_unit =True,
    Repeat_unit_compound_with_ports = compound_with_2_ports_DA, # Here we pass already the repeat unit with ports

    Dummy_atoms = True,
    Dummy_atom_dictionary = dummy_atom_dictionary,

    
    repeat_units = 5,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 15.0,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                              #Crosslinker details
    crosslinker_name = "BASE",                                # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports
    Crosslinker_compound = Dummy_Crosslinker_Polymer,
    

 
                                                              #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 40,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "N",
                            "target_atom" : "F",
                            "bonded_atoms": ["C", "H", "F"], 
                            "anchor_atom_n_bonds" : 4,
                            },
                            
                          
                            ],
    
    
    crosslinker_backbone_target_list = [
                        {
                        "anchor_atom" : "C",
                        "target_atom" : ["S"],
                        "anchor_atom_n_bonds" : 4,
                        "extra_pruning": [],
                        "exclude": [],
                        },
                        
                    
    ],
    
    prune_extra_flag =False,                               # No pruning extra atoms 
    crosslink_sites_per_molecule = 1 ,                     # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
   
    

        
    
    
)

for suffix in ["mol2", "pdb"]:
    box_system.save(f"output/5_3_EPON-862_DETDA-Poly.{suffix}",overwrite = True)

Number of ports per chain 2
This values are the number of crosslinking monomers that will be bonded between each pair of chains to attain the desire degree of crosslinking
{1: 26, 2: 53}
To attain the degree of crosslinking 40% 
 4 crosslinkers are needed 
 2 between each polymer chain
The actual attainable value of crosslinking is 53%
Which is the highest crosslinking percentage attainable
Repeat unit passed to function


Done with loading compounds
{'POL-0': [(0, array([1.42768002, 0.01147   , 0.20259   ])), (1, array([ 2.47937989, -0.18545   ,  0.56428999])), (2, array([ 6.32626118, -0.58465415,  0.6656002 ])), (3, array([ 7.38292086, -0.83222896,  0.97841648])), (4, array([11.1957288 , -1.48200436,  0.88670552])), (5, array([12.25119201, -1.78294811,  1.15336501])), (6, array([16.00259953, -2.69023925,  0.88524613])), (7, array([17.05072264, -3.04670129,  1.10896418])), (8, array([20.71405245, -4.2157275 ,  0.6829175 ])), (9, array([21.74876952, -4.62926965,  0.86736388]))], 'POL-1': [(10, array([1.42768002e+00, 1.14700003e-02, 1.52025900e+01])), (11, array([ 2.47937989, -0.18545   , 15.56428999])), (12, array([ 6.32626118, -0.58465415, 15.6656002 ])), (13, array([ 7.38292086, -0.83222896, 15.97841648])), (14, array([11.1957288 , -1.48200436, 15.88670552])), (15, array([12.25119201, -1.78294811, 16.15336501])), (16, array([16.00259953, -2.69023925, 15.88524613])), (17, array([17.05072264, -3.04670129

In [82]:
box_system.visualize()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.